In [1]:
%load_ext cython

In [13]:
%%cython -a
from itertools import combinations
from constants import NUMERICAL_DECK
import numpy as np
narr = np.arange(27, dtype = np.dtype('i')).reshape((3,3,3))
cdef int [:,:,:] narr_view = narr

    



In [14]:
from constants import LIST_NUMERICAL_DECK
from itertools import combinations
import numpy as np


In [15]:
%%timeit
h = combinations(LIST_NUMERICAL_DECK,5)
card_combinations = np.fromiter(h, dtype = np.dtype((int,(5,2))))

4.05 s ± 182 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
%%cython -a
from constants import LIST_NUMERICAL_DECK
from itertools import combinations
from math import comb
cimport numpy as cnp
import numpy as np

def hand_to_matrix(cards):
    """Pass a list of cards and outputs a matrix representation of those cards."""
    output = np.zeros((13,4), dtype = 'i')
    for card in cards:
        #adjustments need to be made to the indices
        output[card[0]-2,card[1]-1] = 1
    return output

#Define a datatype that allows these arrays to be efficiently passed to cdef functions. 

ctypedef cnp.int_t DTYPE_t

#Define a rank function that operates on these arrays. 

cdef short rank(cnp.ndarray[DTYPE_t, ndim=2] hand_matrix, int[6] results):
    """Pass a 13 by 4 numpy array that represents a 7 card hand.
    For easier indexing, lowest number is highest card.
    Modify the results array in place."""

    #count values and suits. These can be used to define almost every hand.

    cdef cnp.ndarray[DTYPE_t, ndim=1] value_counts = np.sum(hand_matrix, axis = 1)
    cdef cnp.ndarray[DTYPE_t, ndim=1] suit_counts = np.sum(hand_matrix, axis = 0)

    #hand rank variable gets highest priority, plus 5 other potential ranking variables
    #based on card values. 

    cdef short hand_rank
    cdef short c1,c2,c3,c4,c5
    cdef short flush = 0
    cdef short flush_column = -1
    cdef short i, j
    cdef short run = 0
    cdef short top = 0

    #Case of a straight flush.
    if np.amax(suit_counts) >=5:
        flush = 1
    #Check for the straight along a column.
        for j in range(4):
            if suit_counts[j] >=5:
                flush_column = j
                for i in range(13):
                    if hand_matrix[i][j] == 1:
                        if run == 0:
                            top = i
                        run+=1
                    else:
                        run = 0
                    if run == 5:
                        break

                if run == 5 or run == 4 and hand_matrix[j][12] == 1:
                    hand_rank = 0
                    c1 = top
        results[0],results[1] = hand_rank, c1
        return 0
    
    cdef short max_repeats = np.amax(value_counts)

    #Case of a four of a kind.
    
    cdef int c2flag = 0
    if max_repeats == 4:
        hand_rank = 1
        for i in range(13):
            if value_counts[i] == 4:
                c1 = i
                if c2flag:
                    break
            elif value_counts[i] != 0 and not c2flag:
                c2 = i
                c2flag = 1
        results[0],results[1], results[3] = hand_rank,c1,c2
        return 0
    
    #find the second most occurring value to determine repeat based hands.

    cdef short second_max_repeats = 1
    cdef short max_repeats_flag = 0

   ##################ISSUE HERE########
    for i in range(13):
        if value_counts[i] == max_repeats:
            if max_repeats_flag:
                second_max_repeats = max_repeats
                break
            else:
                max_repeats_flag = 1

        elif value_counts[i]>second_max_repeats:
            second_max_repeats = value_counts[i]
#############################################
    #Case of a full house.

    if max_repeats == 3 and second_max_repeats >=2:
        hand_rank = 2
    #Find the best three. 
        for i in range(13):
            if value_counts[i] == 3:
                c1 = i
                break
    #Find the best two.
        for i in range(13):
            if value_counts[i] >= 2 and i != c1:
                c2 = i
                break
        results[0],results[1],results[2] = hand_rank,c1,c2
        return 0
    
    #Check for a flush.
   
    cdef int[5] top_five
    j = 0
    if flush:
        hand_rank = 3
        for i in range(13):
            if j == 5:
                break
            if hand_matrix[i][flush_column] == 1:
                top_five[j] = i
                j+=1
        c1,c2,c3,c4,c5 = top_five
        results[0],results[1],results[2],results[3],results[4],results[5] = hand_rank,c1,c2,c3,c4,c5
        return 0

    #Check for the straight along value counts.
    run = 0
    top = 0
    for i in range(13):
        if value_counts[i] == 1:
            if run == 0:
                top = i
            run+=1
        else:
            run = 0
        if run == 5:
            break

    #Case of a straight.  
           
    if run == 5 or run == 4 and value_counts[12] > 0:
        hand_rank = 4
        c1 = top
        results[0],results[1] = hand_rank,c1
        return 0

    #Case of a three of a kind.

    if max_repeats == 3:
        hand_rank = 5
        #Find the best three.
        for i in range(13):
            if value_counts[i] == 3:
                c1 = i
                break
        #Find the next best two cards. 
        j = 0
        for i in range(13):
            if j == 2:
                break
            if value_counts[i] == 1:
                top_five[j] = i
                j+=1
        c2,c3,_,_,_ = top_five
        results[0],results[1],results[2],results[3] = hand_rank,c1,c2,c3
        return 0

    #Case of a two pair.
    
    if max_repeats == 2 and second_max_repeats == 2:
        hand_rank = 6
        #find the best two and the second best two
        j = 0
        for i in range(13):
            if j == 2:
                break
            if value_counts[i] == 2:
                top_five[j] = i
                j+=1
        c1,c2,_,_,_ = top_five
        #find the best single
        for i in range(13):
            if value_counts[i] == 1:
                c3 = i
                break
        results[0],results[1],results[2],results[3] = hand_rank,c1,c2,c3
        return 0
    
    #Case of pair.

    if max_repeats == 2:
        hand_rank = 7
        #find the best two
        for i in range(13):
            if value_counts[i] == 2:
                c1 = i
                break
        #find the next best three
        j = 0
        for i in range(13):
            if j == 3:
                break
            if value_counts[i] == 1:
                top_five[j] = i
                j+=1
        c2,c3,c4,_,_ = top_five
        results[0],results[1],results[2],results[3],results[4] = hand_rank,c1,c2,c3,c4
        return 0    

    #Case of high card.

    hand_rank = 8
    j = 0
    for i in range(13):
        if j == 5:
            break
        if value_counts[i] == 1:
            top_five[j] = i
            j+=1
    c1,c2,c3,c4,c5 = top_five
    results[0],results[1],results[2],results[3],results[4],results[5] = hand_rank,c1,c2,c3,c4,c5
    return 0
    


def probabilities(community_cards, *holdem_hands):
    """Pass a list of known community cards, and a sequence of pairs corresponding
    to hold-em hands. Outputs a numpy array giving the probability of each of
    the hands being a winning hand or tying hand."""

    #build the deck of remaining cards
    used_cards = set(community_cards) 
    holdem_cards = set()
    for hand in holdem_hands:
        for card in hand:
            holdem_cards.add(card)
    used_cards = used_cards.union(holdem_cards)
    remaining_cards = list(filter(lambda x: tuple(x) not in used_cards, LIST_NUMERICAL_DECK))
    cdef int num_unknown = 5 - len(community_cards)

    #count the number of possible community hands, then build them
    cdef int m = comb(len(remaining_cards),num_unknown)
    cdef tuple c
    community_hands = [c for c in combinations(remaining_cards,num_unknown)]
    converted_hands = [hand_to_matrix(c) for c in community_hands]

def rankp():
    cdef int[6] results
    results = [0,0,0,0,0,0]
    cards = hand_to_matrix([[2,1],[2,2],[2,3],[3,1],[5,3],[4,2],[10,1]])
    rank(cards,results)
    return results




In [81]:
%%timeit
rankp()

18.8 µs ± 382 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
%%cython
L = [1,2,3]
cdef mod(arr):
    arr[0]= 4
    
mod(L)
print(L)

[4, 2, 3]


In [8]:
%%cython
cdef int[2] L = [1,2]
cdef int c1,c2
c1,c2 = L
print(c1,c2)

1 2
